In [1]:
import numpy as np
import pandas as pd
import os
import re #regular expression re Identifiers, and White space characters

In [2]:
!pip install -U textblob#library for processing textual data


ERROR: Invalid requirement: 'textblob#library'
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
data=pd.read_csv("train_phd.csv") #Reading data
type(data) #Type is a data frame
data.head(5) #Displays first five records

,Reviewid,Hotelid,userid,Date,reviewtext,Sentiment
0,Review_1,hotel_101,hotel_1608,"Nov 16, 2007",Nice Marriot View of my king bed room,good
1,Review_2,hotel_101,hotel_6939,"Oct 30, 2007","Good hotel, charges for internet access The Ma...",good
2,Review_3,hotel_101,hotel_3976,"Oct 12, 2007",Small but adequate rooms If you have an early ...,good
3,Review_4,hotel_101,hotel_2851,"Aug 31, 2007","Better than average, some noisy rooms I have s...",good
4,Review_5,hotel_101,hotel_7897,"Jul 18, 2007",Ordinary Although it is highly rated in these ...,bad


In [4]:
type(data)

pandas.core.frame.DataFrame

## transform our reviewtext into lower case.This avoids having multiple copies of the same words

In [5]:
data['reviewtext'] = data['reviewtext'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['reviewtext'].head()

0                nice marriot view of my king bed room
1    good hotel, charges for internet access the ma...
2    small but adequate rooms if you have an early ...
3    better than average, some noisy rooms i have s...
4    ordinary although it is highly rated in these ...
Name: reviewtext, dtype: object

## Removing Punctuation


In [6]:
data['reviewtext'] = data['reviewtext'].str.replace('[^\w\s]','')

##lemmetize

In [7]:
from textblob import Word

## lemmetize

In [8]:
data['reviewtext'] = data['reviewtext'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['reviewtext'].head()

0                nice marriot view of my king bed room
1    good hotel charge for internet access the marr...
2    small but adequate room if you have an early m...
3    better than average some noisy room i have sta...
4    ordinary although it is highly rated in these ...
Name: reviewtext, dtype: object

#stemming 

In [9]:
from nltk.stem import PorterStemmer
st = PorterStemmer()
data['reviewtext'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0                nice marriot view of my king bed room
1    good hotel charg for internet access the marri...
2    small but adequ room if you have an earli morn...
3    better than averag some noisi room i have stay...
4    ordinari although it is highli rate in these r...
Name: reviewtext, dtype: object

## spelling correction

In [10]:
from textblob import TextBlob
data['reviewtext'][:5].apply(lambda x: str(TextBlob(x).correct()))

0                nice married view of my king bed room
1    good hotel charge for internet access the merr...
2    small but adequate room if you have an early m...
3    better than average some noisy room i have sta...
4    ordinary although it is highly rate in these r...
Name: reviewtext, dtype: object

## Stop words removal 

In [11]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['reviewtext'] = data['reviewtext'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data['reviewtext'].head()


0                      nice marriot view king bed room
1    good hotel charge internet access marriott air...
2    small adequate room early morning flight catch...
3    better average noisy room stayed hotel time in...
4    ordinary although highly rated review pay prem...
Name: reviewtext, dtype: object

## Common word removalThis avoids having multiple copies of the same wordsFor example, while calculating the word count, ‘Analytics’ and ‘analytics’ will be taken as different words.

In [12]:
freq = pd.Series(' '.join(data['reviewtext']).split()).value_counts()[:10]
freq

wa          22237
hotel       14179
room        13614
great        5910
stay         4909
would        3902
location     3902
night        3734
good         3705
staff        3558
dtype: int64

## Now, let’s remove these words as their presence will not of any use in classification of our text data.



In [13]:
freq = list(freq.index)
data['reviewtext'] = data['reviewtext'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
data['reviewtext'].head()

0                           nice marriot view king bed
1    charge internet access marriott airport seattl...
2    small adequate early morning flight catch choi...
3    better average noisy stayed time inthe last se...
4    ordinary although highly rated review pay prem...
Name: reviewtext, dtype: object

In [14]:
Text_data = data['reviewtext']

In [15]:
Text_data1 = Text_data.tolist()

## Rare words removal  this time let’s remove rarely occurring words from the text. Because they’re so rare, the association between them and other words is dominated by noise

In [16]:
freq_rare= pd.Series(' '.join(data['reviewtext']).split()).value_counts()[-10:]
freq_rare

disappointedits    1
minumum            1
terriblelot        1
5oh                1
endwould           1
updatedwell        1
timewarped         1
patina             1
sometimesmy        1
watchif            1
dtype: int64

In [17]:
freq_rare = list(freq_rare.index)
data['reviewtext'] = data['reviewtext'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
data['reviewtext'].head()

0                           nice marriot view king bed
1    charge internet access marriott airport seattl...
2    small adequate early morning flight catch choi...
3    better average noisy stayed time inthe last se...
4    ordinary although highly rated review pay prem...
Name: reviewtext, dtype: object

In [18]:
Text_data = data['reviewtext']

In [19]:
Text_data1 = Text_data.tolist()

In [20]:
type(Text_data1)

list

## removing white spaces and numbers # it has to be executed after converting to list 

In [21]:
for i in range(len(data)):
    Text_data1[i]=re.sub(r'\s+', ' ', Text_data1[i]) #Removing more than one white spaces in the sentence
    Text_data1[i]=re.sub('[\d]', '',Text_data1[i])#Removing numbers

In [22]:
Text_data1

['nice marriot view king bed',
 'charge internet access marriott airport seattle reasonable choice seatac area paid  per shuttle generally quick airport encourage use rather taxi service spotty seattle second floor back long walk matter way took interior nice check quickbreakfast restaurant bit pricey drink service bar fine complaint imagine must get noisy lobby crowd hanging thebar guess marriott ha new program guest charged internet service  free business center near checkin counter two internet pc one boarding pas access pcpersonally ability sit check personal work email nice sure many taker pay extra access hoping get nickel dimed thati offer four star overall quality',
 'small adequate early morning flight catch choice right airport ha recently renovated new clean service fast efficient food negative small bathroom smallest ive encountered many year hardly turn around im small person however countertop granite nice also one elevator seemed odd size may self park valet park fee rec

# Split the data into train and test 

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train, y_test=train_test_split(Text_data1,data['Sentiment'],test_size=0.3,random_state=555)

### Construct tf-idf matrix

In [25]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [26]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [27]:
count=CountVectorizer(strip_accents='unicode',ngram_range=(1,2),stop_words='english')


In [28]:
tfreq=count.fit_transform(Text_data1)

In [29]:
Dense_mat=tfreq.todense()

In [30]:
A=pd.DataFrame(Dense_mat,columns=count.get_feature_names())

## Lets construct TF-idf matrix based on the train documents

In [31]:
tfidf_transformer = TfidfVectorizer(ngram_range=(1,1),stop_words='english')

In [32]:
count=CountVectorizer(strip_accents='unicode',ngram_range=(1,2),stop_words='english')


In [33]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train)

In [34]:
X_train_tfidf.shape

(4400, 22358)

In [35]:
# Get the tfidf matrix for test documents
X_test_tfidf = tfidf_transformer.transform(X_test)
X_test_tfidf.shape
X_test_tfidf.shape

(1887, 22358)

In [36]:
print(X_train_tfidf)

  (0, 2747)	0.0553929702307985
  (0, 3753)	0.1170320567869043
  (0, 2827)	0.1408568667071144
  (0, 5958)	0.17008692671998554
  (0, 8911)	0.06989294736278731
  (0, 7330)	0.0732821052905102
  (0, 14404)	0.07411212023829962
  (0, 16628)	0.2099290360772506
  (0, 1021)	0.04004008983594123
  (0, 17982)	0.07227329675121488
  (0, 11036)	0.10925930068078975
  (0, 15477)	0.11916983276834578
  (0, 21827)	0.11054074749708397
  (0, 11286)	0.05695720887104299
  (0, 14868)	0.05869099987917735
  (0, 8819)	0.07010499733140076
  (0, 17473)	0.0758151986057802
  (0, 1649)	0.08973982644041856
  (0, 2977)	0.04602541222515768
  (0, 13932)	0.04485932027751007
  (0, 11313)	0.047463767877352916
  (0, 12755)	0.08587893318853847
  (0, 1133)	0.05420349475487775
  (0, 17171)	0.07422017082370529
  (0, 8626)	0.05644892686061126
  :	:
  (4398, 21356)	0.155661457748139
  (4398, 19811)	0.15604088114648834
  (4398, 9202)	0.09578088928365201
  (4398, 10020)	0.10478775076373352
  (4398, 21)	0.08429697062350934
  (4398, 142

### Implementing Naive Bayes Classification model

In [37]:
from sklearn.naive_bayes import MultinomialNB
nb_clf = MultinomialNB().fit(X_train_tfidf, y_train) #nb_clf is the classifier
pred_train = nb_clf.predict(X_train_tfidf) #get predictions on train data


In [38]:
from sklearn.naive_bayes import MultinomialNB
nb_clf = MultinomialNB().fit(X_train_tfidf, y_train) #nb_clf is the classifier
pred_train = nb_clf.predict(X_train_tfidf) #get predictions on train data

from sklearn.metrics import confusion_matrix
prediction=confusion_matrix(y_train, pred_train) #get confusion matrix for train predictions

In [39]:
nb_clf = MultinomialNB().fit(X_train_tfidf, y_train) #nb_clf is the classifier


In [40]:
print("Train_Confusion Matrix:\n",prediction,"\n")

from sklearn.metrics import recall_score, precision_score, accuracy_score

acc_train = accuracy_score(y_train, pred_train)

print("Results of Naive Bayes on train data:","\nAccuracy:",acc_train)

Train_Confusion Matrix:
 [[  21    0  724]
 [   0   97 1273]
 [   0    0 2285]] 

Results of Naive Bayes on train data: 
Accuracy: 0.5461363636363636


In [41]:
pred = nb_clf.predict(X_test_tfidf) #predict on test data
prediction = confusion_matrix(y_test,pred) ##get confusion matrix for test predictions

In [42]:
print("Test_Confusion matrix:\n", prediction,"\n")
acc = accuracy_score(y_test,pred)

print("Results of Naive Bayes on test data:","\nAcc: ",acc)

Test_Confusion matrix:
 [[  1   0 357]
 [  0  14 587]
 [  0   2 926]] 

Results of Naive Bayes on test data: 
Acc:  0.49867514573396926


In [43]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,roc_curve

In [44]:
from sklearn.metrics import confusion_matrix

## Accuracy on naive bayes is 0.49867514573396926

## Implementing Logistic Regression

In [45]:
from sklearn import linear_model

logreg_train = linear_model.LogisticRegression()
lr_clf = logreg_train.fit(X_train_tfidf, y_train) #fit logreg model on train data

pred_lr_train=lr_clf.predict(X_train_tfidf) # predict on train data
pred_lr_test=lr_clf.predict(X_test_tfidf) # predict on test data


print("Train_Confusion matrix:\n", confusion_matrix(y_train,pred_lr_train))
print("Test_Confusion matrix:\n",confusion_matrix(y_test,pred_lr_test))

Train_Confusion matrix:
 [[ 526    2  217]
 [   2  947  421]
 [  22  113 2150]]
Test_Confusion matrix:
 [[194   4 160]
 [  2 288 311]
 [ 18 114 796]]


In [46]:
print("Results of Logistic regression on train data")
acc=accuracy_score(y_train,pred_lr_train)
print("\nAcc: ",acc)


print("Results of Logistic regression on test data")
acc=accuracy_score(y_test,pred_lr_test)

print("\nAcc: ",acc)

Results of Logistic regression on train data

Acc:  0.8234090909090909
Results of Logistic regression on test data

Acc:  0.6772655007949125


## Accuracy on loistic regression 0.6772655007949125

In [47]:
logistic = linear_model.LogisticRegression()

In [48]:
penalty = ['l1', 'l2']
C = np.logspace(0, 4, 10)
hyperparameters = dict(C=C, penalty=penalty)

## logistic regression GRID SEARCH 

In [49]:
from sklearn.model_selection import GridSearchCV

In [50]:
from sklearn import linear_model

In [51]:
logistic = linear_model.LogisticRegression()

In [52]:
penalty = ['l1', 'l2']
C = np.logspace(0, 4, 10)
hyperparameters = dict(C=C, penalty=penalty)

In [53]:
clf_grid = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)


In [54]:
best_model = clf_grid.fit(X_train_tfidf, y_train)


In [55]:
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])


Best Penalty: l1
Best C: 2.7825594022071245


In [56]:
logistic_final = linear_model.LogisticRegression(penalty="l2", C = 2.7825594022071245)


In [57]:
logistic_final.fit(X_train_tfidf, y_train)


LogisticRegression(C=2.7825594022071245, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [58]:
from sklearn.metrics import accuracy_score

In [59]:
pred_final_train=logistic_final.predict(X_train_tfidf) # predict on train data

In [60]:
pred_final_train=logistic_final.predict(X_train_tfidf) # predict on train data

In [61]:
pred_final_test=logistic_final.predict(X_test_tfidf)#predict on test data 

In [62]:
print("Results of Logistic regression on train data")
acc=accuracy_score(y_train,pred_final_train)

print("\nAcc: ",acc)

print("Results of Logistic regression on test data")
acc=accuracy_score(y_test,pred_final_test)

print("\nAcc: ",acc)

Results of Logistic regression on train data

Acc:  0.9118181818181819
Results of Logistic regression on test data

Acc:  0.6873343932167462


In [63]:
##Accuracy on logistic regression using grid search best parameters 0.Acc:  0.690779014308426

In [64]:
pred_final_train=logistic_final.predict(X_train_tfidf) # predict on train data

In [65]:
pred_final_train=logistic_final.predict(X_test_tfidf) # predict on train data

In [66]:
from sklearn.ensemble import RandomForestClassifier

In [67]:
rfc_mod=RandomForestClassifier(n_estimators=50,oob_score=True)

In [68]:
rfc_mod.fit(X=X_train_tfidf,y=y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [69]:
pred_train_rf=rfc_mod.predict(X_train_tfidf)


In [70]:
print("Accuracy on train is:",accuracy_score(y_train,pred_train_rf))


Accuracy on train is: 0.9977272727272727


In [71]:
pred_test_rf=rfc_mod.predict(X_test_tfidf)

In [72]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,roc_curve

In [73]:
print("Accuracy on test is:",accuracy_score(y_test,pred_test_rf))


Accuracy on test is: 0.604133545310016


In [74]:
##Accuracyon random forest classifier is 0.6208267090620032

## XGBOOST

In [75]:
from xgboost import XGBClassifier
XGB_model = XGBClassifier()

In [ ]:
XGB_model_train= XGB_model.fit(X=X_train_tfidf,y=y_train)

In [ ]:
%time XGB_model.fit(X=X_train_tfidf,y=y_train)

In [ ]:
pred_train_XG = XGB_model.predict(X_train_tfidf)
pred_test_XG = XGB_model.predict(X_test_tfidf)


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,roc_curve

In [ ]:
print("Accuracy on train is:",accuracy_score(y_train,pred_train_XG ))
print("Accuracy on test is:",accuracy_score(y_test,pred_test_XG ))
print(classification_report(y_true = y_train,y_pred = pred_train_XG ))
print(classification_report(y_true = y_test,y_pred = pred_test_XG ))

In [ ]:
##Accuracy using XGBOOST is 0.0.6422893481717011

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc

In [ ]:
svc_c5_rbf = SVC(C=5,kernel='rbf')
svc_c5_rbf

In [ ]:
svc_c5_rbf.fit(X=X_train_tfidf,y=y_train)

In [ ]:
pred_svc = svc_c5_rbf.predict(X_train_tfidf)

In [ ]:
pred_svc_test=svc_c5_rbf.predict(X_test_tfidf)

In [ ]:
accuracy_score(y_test,pred_svc_test)

## Accuracy using SVC is 0.0.4917859035506094

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
svc_grid = SVC(random_state=555)

In [ ]:
param_grid = {

'C': [0.001, 0.01, 0.1, 1, 10],
'gamma': [0.001, 0.01, 0.1, 1], 
'kernel':['linear', 'poly', 'rbf', 'sigmoid']}

 
svc_cv_grid = GridSearchCV(estimator = svc_grid, param_grid = param_grid, cv = 3)

In [ ]:
svc_cv_grid.fit(X = X_train_tfidf, y = y_train)

In [ ]:
print(svc_cv_grid.best_score_,svc_cv_grid.best_params_)

In [ ]:
pred_train = svc_cv_grid.predict(X_train_tfidf)

In [ ]:
pred_test = svc_cv_grid.predict(X_test_tfidf)

In [ ]:
print(y_train.shape)
print(X_train_tfidf.shape)

In [ ]:
print(accuracy_score(y_train,pred_train))

In [ ]:
print(accuracy_score(y_test,pred_test))

In [ ]:
new_test=pd.read_csv("Test-1555730055539.csv")

In [ ]:
new_test.head()

## gradient boosting 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GBM_model = GradientBoostingClassifier(n_estimators=50,
                                       learning_rate=0.3,
                                       subsample=0.8)

In [ ]:
GBM_model.fit(X=X_train_tfidf,y=y_train)

In [ ]:
pred_test_GB = GBM_model.predict(X_test_tfidf)
pred_train_GB = GBM_model.predict(X_train_tfidf)
print("Accuracy on train is:",accuracy_score(y_train,pred_train_GB ))
print("Accuracy on test is:",accuracy_score(y_test,pred_test_GB ))
print(classification_report(y_train,pred_train_GB ))
print(classification_report(y_test,pred_test_GB ))

## Decision trees

In [ ]:
import graphviz
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score
from sklearn import tree

In [ ]:
dtc = DecisionTreeClassifier()

In [ ]:
dtc.fit(X=X_train_tfidf,y=y_train)

In [ ]:
pred_train = dtc.predict(X_train_tfidf)
pred_test = dtc.predict(X_test_tfidf)
print("Accuracy on train is:",accuracy_score(y_train,pred_train))
print("Accuracy on test is:",accuracy_score(y_test,pred_test))
print(classification_report(y_train,pred_train))
print(classification_report(y_test,pred_test))

In [ ]:
##Accuracy using Decisiontrees is 0.5246422893481717 

## PREDICT ON NEW TEST DATA 

In [ ]:
data1=pd.read_csv("Test-1555730055539.csv")

In [ ]:
data1.head()

## Removing Punctuation


In [ ]:
data1['reviewtext'] = data1['reviewtext'].str.replace('[^\w\s]','')

## stop words removal 

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data1['reviewtext'] = data1['reviewtext'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data1['reviewtext'].head()


## common words removal 

In [ ]:
freq = pd.Series(' '.join(data1['reviewtext']).split()).value_counts()[:10]
freq

In [ ]:
freq = list(freq.index)
data1['reviewtext'] = data1['reviewtext'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
data1['reviewtext'].head()


## transform review text to lower case 

In [ ]:
data1['reviewtext'] = data1['reviewtext'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data1['reviewtext'].head()

In [ ]:
##spelling correction 

In [ ]:
from textblob import TextBlob
data1['reviewtext'][:5].apply(lambda x: str(TextBlob(x).correct()))

## Rare words removal

In [ ]:
freq_rare_test= pd.Series(' '.join(data1['reviewtext']).split()).value_counts()[-10:]
freq_rare_test

In [ ]:
freq_rare_test = list(freq_rare_test.index)
data1['reviewtext'] = data1['reviewtext'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
data1['reviewtext'].head()

## lemmetize

In [ ]:
from textblob import Word

In [ ]:
data1['reviewtext'] = data1['reviewtext'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data1['reviewtext'].head()

## spelling correction

In [ ]:
from textblob import TextBlob
data['reviewtext'][:5].apply(lambda x: str(TextBlob(x).correct()))

In [ ]:
## Stop words removal 

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data1['reviewtext'] = data1['reviewtext'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data1['reviewtext'].head()


## stemming

In [ ]:
from nltk.stem import PorterStemmer
st = PorterStemmer()
data1['reviewtext'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [ ]:
data1.shape

In [ ]:
##lemmetize

In [ ]:
data1['reviewtext'] = data1['reviewtext'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data1['reviewtext'].head()

In [ ]:
data2=data1["reviewtext"]

In [ ]:
data3=data2.tolist()

## Removing white spaces and numbers 

In [ ]:
for i in range(len(data3)):
    data3[i]=re.sub(r'\s+', ' ', data3[i]) #Removing more than one white spaces in the sentence
    data3[i]=re.sub('[\d]', '',data3[i]) #Removing numbers

In [ ]:
data3_tfidf = tfidf_transformer.transform(data3)
data3_tfidf.shape

In [ ]:
new_pred=svc_cv_grid.predict(data3_tfidf)

In [ ]:
data1.shape

In [ ]:
new_pred.size

In [ ]:
data1.shape

In [ ]:
pred5=pd.DataFrame()

In [ ]:
data1['Sentiment'] = new_pred

In [ ]:
data1.shape

In [ ]:
data1[['Reviewid','Sentiment']].to_csv("Review_svc_cv_grid.csv",index = False)

# #predictions on test and upload  to sct 

In [ ]:
predictions_lr_grid = pd.DataFrame()

In [ ]:
predictions_lr_grid["ReviewID"] = new_test3["Reviewid"]

In [ ]:
predictions["Sentiment"] = new_pred

In [ ]:
predictions.to_csv("Review_predcitions.csv")

In [ ]:
pred2 =lr_clf.predict(new_test2_tfidf)

In [ ]:
predictions_lr_newtest2 = pd.DataFrame()

In [ ]:
predictions_lr_newtest2["ReviewID"] = new_test["Reviewid"]

In [ ]:
predictions_lr_newtest2["Sentiment"]=pred2

In [ ]:
predictions_lr_newtest2.to_csv("Review_predictions_lr.csv")